## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [489]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [490]:
db = debar = pd.read_csv('../pset2/debar.csv')
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [491]:
## number of rows
print("The number of rows in the debar data is: " , len(debar)) 

## number of unique employer names 
print("The number of unique employer names is: " , debar.Name.nunique())

## answer
print('There does not appear to be one row per employer, but multiple rows for some employers.')

The number of rows in the debar data is:  114
The number of unique employer names is:  98
There does not appear to be one row per employer, but multiple rows for some employers.


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [492]:
## A. create is_repeated column using value_counts. 

name_counts= db['Name'].value_counts()
db['is_repeated'] = db['Name'].map(name_counts) > 1

## B. print the rows when is_repeated == True 
print(db[db['is_repeated']==True])

## interpret: 
print("The hypothesis from part 1.1 is supported by the values contained in the is_repeated column\nIt is now clear that there are multiple rows for some employers.")

## C. subset to when is_repeated == True
mdb = mult_debar = db[db['is_repeated']==True]
print(mult_debar.head())
print(mult_debar.shape)

                                        Name       City, State  \
6                    Annabella Land & Cattle     Annabella, UT   
7                        Autumn Hill Orchard        Groton, MA   
8      Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
11                     Loewen Harvesting LLC   Brownsville, TX   
12               Rollo Farm Labor Contractor         Miami, FL   
14                             Sharon Mathis        Tifton, GA   
15                                 SRT Farms        Morton, TX   
16                               Mark Duncan     Roosevelt, UT   
17          Maple Ridge Custom Services, LLC     Altheimer, AK   
18                                 F&W Farms       Ingalls, KS   
19                        Cisco Produce Inc.         Cairo, GA   
21   Old Tree Farms/Verpaalen Custom Service         Volga, SD   
24               Rollo Farm Labor Contractor         Miami, FL   
25                     Loewen Harvesting LLC    Brownfield, TX   
28     Cad

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [493]:
## convert Violation to lowercase 
mdb.loc[:, 'Violation'] = mdb['Violation'].str.lower()
mdb['Violation']


## fix issue with City, State names -- standardize format. 
mdb.loc[:, 'City, State'] = mdb['City, State'].replace(['Caddo, Texas','Annabella, Utah', 'Lyons, Georgia'], 
                                                    ['Caddo, TX', 'Annabella, UT','Lyons, GA' ]) 
mdb['City, State']

mdb['Identifier'] = mdb['Name'] + ', '   + mdb['City, State'] 

mdb_sorted = mdb.sort_values(by = ['Identifier', 'Start date']) 

mdb_sorted['Violation_Number'] = mdb_sorted.groupby('Identifier').cumcount() + 1 
mdb_sorted
mult_debar_wide = mdb_sorted.pivot(index = 'Identifier', columns = ['Violation_Number'] , values = 'Start date')


# ## create a pivot table with Name and City, State as the columns and index, with Start date as the values,
# ## and create two new columns to mark the first and second start date, to identify the duplicates 
# mult_debar_wide = mdb.pivot_table(index=['Name', 'City, State'], 
#                                   ## assign numeric value to each row that we can access 
#                                   columns=mdb.groupby(['Name', 'City, State', 'Start date']).cumcount() + 1, 
#                                   values='Start date', 
#                                   ## select first 2 
#                                   aggfunc=lambda x: x.head(2)).reset_index()

# mult_debar_wide = mult_debar_wide.rename(columns =  {0:'Identifier', 1:'start_date_viol1', 2:'start_date_viol2'})

mult_debar_wide.columns= [f'start_date_viol{col}' for col in mult_debar_wide.columns]
print(mult_debar_wide.head() )
print(mult_debar_wide.shape)


mult_debar_wide


6                                            non payment
7              failure to respond to audit (no response)
8         failure to respond to audit (partial response)
11        failure to respond to audit (partial response)
12             failure to respond to audit (no response)
14             failure to respond to audit (no response)
15             failure to respond to audit (no response)
16             failure to respond to audit (no response)
17        failure to respond to audit (partial response)
18        failure to respond to audit (partial response)
19             failure to respond to audit (no response)
21                                         whd debarment
24            impeding the audit process – non- response
25        impeding the audit process – partial- response
28        impeding the audit process – partial- response
29            impeding the audit process – non- response
30                                           non-payment
31             failure to respo

6         Annabella, UT
7            Groton, MA
8             Caddo, TX
11      Brownsville, TX
12            Miami, FL
14           Tifton, GA
15           Morton, TX
16        Roosevelt, UT
17        Altheimer, AK
18          Ingalls, KS
19            Cairo, GA
21            Volga, SD
24            Miami, FL
25       Brownfield, TX
28            Caddo, TX
29           Groton, MA
30        Annabella, UT
31           Plains, TX
51            Volga, SD
55           Plains, TX
56            Cairo, GA
58          Ingalls, KS
59        Altheimer, AR
60        Roosevelt, UT
61           Morton, TX
62           Tifton, GA
73            Lyons, GA
89            Lyons, GA
103    Mount Vernon, TX
106           Healy, KS
109    Mount Vernon, TX
111           Healy, KS
Name: City, State, dtype: object

/var/folders/3f/t59vtj796q5bjl4slmbv42f40000gn/T/ipykernel_48535/4144945370.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdb['Identifier'] = mdb['Name'] + ', '   + mdb['City, State']


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,Identifier,Violation_Number
6,Annabella Land & Cattle,"Annabella, UT",non payment,1 year,5/9/2014,5/9/2015,True,"Annabella Land & Cattle, Annabella, UT",1
30,Annabella Land & Cattle,"Annabella, UT",non-payment,1 year,5/9/2014,5/8/2015,True,"Annabella Land & Cattle, Annabella, UT",2
7,Autumn Hill Orchard,"Groton, MA",failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True,"Autumn Hill Orchard, Groton, MA",1
29,Autumn Hill Orchard,"Groton, MA",impeding the audit process – non- response,2 years,7/6/2014,7/5/2016,True,"Autumn Hill Orchard, Groton, MA",2
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True,"Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX",1
28,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",impeding the audit process – partial- response,2 years,7/20/2014,7/19/2016,True,"Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX",2
19,Cisco Produce Inc.,"Cairo, GA",failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,True,"Cisco Produce Inc., Cairo, GA",1
56,Cisco Produce Inc.,"Cairo, GA",impeding the audit process – non- response,2 years,12/10/2015,12/9/2017,True,"Cisco Produce Inc., Cairo, GA",2
103,Dove Creek Farms,"Mount Vernon, TX",failure to respond to audit request,2 years,2/9/2018,2/8/2018,True,"Dove Creek Farms, Mount Vernon, TX",1
109,Dove Creek Farms,"Mount Vernon, TX",failure to respond to audit request,2 years,2/9/2018,2/8/2020,True,"Dove Creek Farms, Mount Vernon, TX",2


                                                 start_date_viol1  \
Identifier                                                          
Annabella Land & Cattle, Annabella, UT                   5/9/2014   
Autumn Hill Orchard, Groton, MA                          7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX        7/20/2014   
Cisco Produce Inc., Cairo, GA                          12/10/2014   
Dove Creek Farms, Mount Vernon, TX                       2/9/2018   

                                                 start_date_viol2  
Identifier                                                         
Annabella Land & Cattle, Annabella, UT                   5/9/2014  
Autumn Hill Orchard, Groton, MA                          7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX        7/20/2014  
Cisco Produce Inc., Cairo, GA                          12/10/2015  
Dove Creek Farms, Mount Vernon, TX                       2/9/2018  
(18, 2)


,start_date_viol1,start_date_viol2
Identifier,,
"Annabella Land & Cattle, Annabella, UT",5/9/2014,5/9/2014
"Autumn Hill Orchard, Groton, MA",7/6/2014,7/6/2014
"Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX",7/20/2014,7/20/2014
"Cisco Produce Inc., Cairo, GA",12/10/2014,12/10/2015
"Dove Creek Farms, Mount Vernon, TX",2/9/2018,2/9/2018
"F&W Farms, Ingalls, KS",12/10/2014,12/10/2014
"Loewen Harvesting LLC, Brownfield, TX",8/20/2014,NaN
"Loewen Harvesting LLC, Brownsville, TX",8/20/2014,NaN
"Macky and Brad Farms, Plains, TX",2/13/2015,2/13/2015


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [494]:
## Add identifier to the debar data 
debar['Identifier'] = debar['Name'] + ', '   + debar['City, State'] 
debar.columns
mult_debar_wide.columns

## create is_dup 
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']
mult_debar_wide.head()

## subset True 
identifies_with_duplicates = mult_debar_wide[mult_debar_wide['is_dup']].index 

## for those that are dup keep vio number = 1 
keep_mask = debar['Identifier'].isin(identifies_with_duplicates)
keep_all_others = ~debar['Identifier'].isin(identifies_with_duplicates)



# ## debar_clean 
debar_clean = pd.concat([debar[keep_mask], debar[keep_all_others]])
debar_clean.drop(columns  = 'is_repeated', inplace = True)
debar_clean


print(debar_clean['Name'].nunique())
print(debar_clean.shape)

Index(['Name', 'City, State', 'Violation', 'Duration', 'Start date',
       'End date', 'is_repeated', 'Identifier'],
      dtype='object')

Index(['start_date_viol1', 'start_date_viol2'], dtype='object')

,start_date_viol1,start_date_viol2,is_dup
Identifier,,,
"Annabella Land & Cattle, Annabella, UT",5/9/2014,5/9/2014,True
"Autumn Hill Orchard, Groton, MA",7/6/2014,7/6/2014,True
"Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX",7/20/2014,7/20/2014,True
"Cisco Produce Inc., Cairo, GA",12/10/2014,12/10/2015,False
"Dove Creek Farms, Mount Vernon, TX",2/9/2018,2/9/2018,True


,Name,"City, State",Violation,Duration,Start date,End date,Identifier
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,"Annabella Land & Cattle, Annabella, UT"
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,"Autumn Hill Orchard, Groton, MA"
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,"Caddo Creek Ranch, dba Paradise Ranch, Caddo, TX"
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,"Rollo Farm Labor Contractor, Miami, FL"
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,"Sharon Mathis, Tifton, GA"
...,...,...,...,...,...,...,...
107,Walker Place,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,"Walker Place, Danville, IL"
108,County Fair Farm (company) and Andrew Williams...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,County Fair Farm (company) and Andrew Williams...
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,"Jesus Ledesma, Mulberry, FL"
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,"B & R Harvesting and Paul Cruz (individual), C..."


98
(114, 7)


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [495]:
# your code here to load the data 
jb = jobs = pd.read_csv('../pset2/jobs.csv')
jb.head()


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [496]:
## merge the two data sets 

jobs['EMPLOYER_NAME']
debar_clean['Name']
merged = pd.merge(jb, debar_clean, left_on=['EMPLOYER_NAME'], right_on=[ 'Name'], how='inner')
merged
print('1 exact match!')

0       Fazio Farms Operating Company, LLC
1                       Charlie Sunderland
2                        Michael Rudebusch
3                             Lodahl Farms
4                  Dunson Harvesting, Inc.
                       ...                
2715                     James L Schneller
2716                       Stephen G Myers
2717                            Andy Povey
2718                Silver Creek Seed, LLC
2719                        Halabura Farms
Name: EMPLOYER_NAME, Length: 2720, dtype: object

6                                Annabella Land & Cattle
7                                    Autumn Hill Orchard
8                  Caddo Creek Ranch, dba Paradise Ranch
12                           Rollo Farm Labor Contractor
14                                         Sharon Mathis
                             ...                        
107                                         Walker Place
108    County Fair Farm (company) and Andrew Williams...
110                                        Jesus Ledesma
112          B & R Harvesting and Paul Cruz (individual)
113                                          Delia Rojas
Name: Name, Length: 114, dtype: object

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,Identifier
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,NaN,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,"Rafael Barajas, Sebring, Florida"


1 exact match!


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [497]:
## insert your code to turn into uppercase here
jb.loc[:,'EMPLOYER_NAME'] = list(map(lambda x: x.upper(), jb['EMPLOYER_NAME']))
debar_clean.loc[:,'Name'] = list(map(lambda x: x.upper(), debar_clean['Name']))
jb['EMPLOYER_NAME'].sample(n=15)
debar_clean['Name'].sample(n=15)

1930                   HOME CYPRESS FARMS
940                   BONNIE PLANTS, INC.
1223                         BOUPROP INC 
746         LANCE & LISA FUNK PARTNERSHIP
1378               TEXAS BEST HONEY, INC.
215                FOOTHILL PACKING, INC.
1935                  TOWNLEY TOMATO FARM
2170               1879 ROBINSON FARM LLC
879                         SETH CHAPPELL
2101                   CHAD MARTIN THEVIS
459                     NEAL LANDRY FARMS
601                             JOHN IHRY
811                 TODD OTAHAL FARMS LLC
1328                      GEL WILLOW CORP
1069    LOUISIANA AQUACULTURE INVESTMENTS
Name: EMPLOYER_NAME, dtype: object

31            MACKY AND BRAD FARMS
56              CISCO PRODUCE INC.
84            LANDMARK LANDSCAPING
25           LOEWEN HARVESTING LLC
81                     OLSON FARMS
77           MYRKA MIREYA CARDENAS
0                   J&J HARVESTING
12     ROLLO FARM LABOR CONTRACTOR
92             JAVIER XISTO ZAPATA
109               DOVE CREEK FARMS
113                    DELIA ROJAS
35                     IRISH FLATS
111                   TURNER FARMS
15                       SRT FARMS
70        JOHN & NETA LEOPKY FARMS
Name: Name, dtype: object

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [498]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [499]:
## insert your code here with the regex pattern for part A
pos_example_1 = "CISCO PRODUCE INC."
pattern = r"(INC\.|CO\.|LLC\.)"

## insert your code to use re.sub to apply the pattern to the test cases for part B
pract_1 = re.sub(pattern, lambda x: x.group().replace('.', ''), pos_example_1);pract_1
pract_2 = re.sub(pattern, lambda x: x.group().replace('.', ''), pos_example_2); pract_2
pract_3 = re.sub(pattern, lambda x: x.group().replace('.', ''), neg_example); pract_3





'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [500]:
jb['name_clean'] = [re.sub(pattern, lambda x: x.group().replace('.', ''), name)
                         for name in jb.EMPLOYER_NAME]
jb['name_clean']
debar_clean['name_clean'] = [re.sub(pattern, lambda x: x.group().replace('.', ''), name)
                         for name in debar_clean.Name]
debar_clean['name_clean']

0       FAZIO FARMS OPERATING COMPANY, LLC
1                       CHARLIE SUNDERLAND
2                        MICHAEL RUDEBUSCH
3                             LODAHL FARMS
4                   DUNSON HARVESTING, INC
                       ...                
2715                     JAMES L SCHNELLER
2716                       STEPHEN G MYERS
2717                            ANDY POVEY
2718                SILVER CREEK SEED, LLC
2719                        HALABURA FARMS
Name: name_clean, Length: 2720, dtype: object

6                                ANNABELLA LAND & CATTLE
7                                    AUTUMN HILL ORCHARD
8                  CADDO CREEK RANCH, DBA PARADISE RANCH
12                           ROLLO FARM LABOR CONTRACTOR
14                                         SHARON MATHIS
                             ...                        
107                                         WALKER PLACE
108    COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...
110                                        JESUS LEDESMA
112          B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
113                                          DELIA ROJAS
Name: name_clean, Length: 114, dtype: object

In [501]:
## subset to only those names which changed
debar_changed_names = debar_clean[debar_clean['name_clean']!=debar_clean['Name']] 
jb_changed_names = jb[jb['name_clean']!=jb['EMPLOYER_NAME']] 
debar_changed_names
(debar_clean[['name_clean', 'Name']])
(jb[['name_clean','EMPLOYER_NAME']])

,Name,"City, State",Violation,Duration,Start date,End date,Identifier,name_clean
3,ANTON FERTILIZER INC.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,"Anton Fertilizer Inc., Dighton, KS",ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,"Great Plains Fluid Service, Inc., Greensburg, KS","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,"Whitewater, KS",Failure to Hire U.S. workers,2 years,5/15/2014,5/14/2016,"Promax Inc., Whitewater, KS",PROMAX INC
13,REIMER'S INC.,"Lakin, KS",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,"Reimer's Inc., Lakin, KS",REIMER'S INC
19,CISCO PRODUCE INC.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,"Cisco Produce Inc., Cairo, GA",CISCO PRODUCE INC
23,REIMER’S INC.,"Lakin, Kansas",Impeding the Audit Process – Non- Response,2 years,8/23/2014,8/22/2016,"Reimer’s Inc., Lakin, Kansas",REIMER’S INC
33,GERONIMO SHEEP CO.,"Vernal, UT",WHD Debarment,3 years,6/23/2014,6/22/2017,"Geronimo Sheep Co., Vernal, UT",GERONIMO SHEEP CO
46,ALTENDORF TRANSPORT INC.,"Minto, ND",Impeding the Audit Process – Non- Response,2 years,4/24/2015,4/23/2017,"Altendorf Transport Inc., Minto, ND",ALTENDORF TRANSPORT INC
56,CISCO PRODUCE INC.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,"Cisco Produce Inc., Cairo, GA",CISCO PRODUCE INC
72,"SAXTONS RIVER ORCHARDS, INC.","Saxtons River, Vermont",Failure to respond to audit request,2 years,7/9/2016,7/8/2018,"Saxtons River Orchards, Inc., Saxtons River, V...","SAXTONS RIVER ORCHARDS, INC"


,name_clean,Name
6,ANNABELLA LAND & CATTLE,ANNABELLA LAND & CATTLE
7,AUTUMN HILL ORCHARD,AUTUMN HILL ORCHARD
8,"CADDO CREEK RANCH, DBA PARADISE RANCH","CADDO CREEK RANCH, DBA PARADISE RANCH"
12,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR
14,SHARON MATHIS,SHARON MATHIS
...,...,...
107,WALKER PLACE,WALKER PLACE
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...
110,JESUS LEDESMA,JESUS LEDESMA
112,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)


,name_clean,EMPLOYER_NAME
0,"FAZIO FARMS OPERATING COMPANY, LLC","FAZIO FARMS OPERATING COMPANY, LLC"
1,CHARLIE SUNDERLAND,CHARLIE SUNDERLAND
2,MICHAEL RUDEBUSCH,MICHAEL RUDEBUSCH
3,LODAHL FARMS,LODAHL FARMS
4,"DUNSON HARVESTING, INC","DUNSON HARVESTING, INC."
...,...,...
2715,JAMES L SCHNELLER,JAMES L SCHNELLER
2716,STEPHEN G MYERS,STEPHEN G MYERS
2717,ANDY POVEY,ANDY POVEY
2718,"SILVER CREEK SEED, LLC","SILVER CREEK SEED, LLC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [502]:
## your code here

jb['name_clean']
debar_clean['name_clean']
second_merged =  pd.merge(jb, debar_clean, on=['name_clean'],how='inner')

second_merged
print('1 match') 

## Fix Slash ranch 
pattern = r'SLASH\s*E\.V\.\s*RANCH\s*LLP'
debar_clean['name_clean_2'] = [re.sub(pattern, 'SLASH EV RANCH', name) 
                    for name in debar_clean.name_clean] 
debar_clean['name_clean_2']

third_merged =  pd.merge(jb, debar_clean, left_on=['name_clean'], right_on=['name_clean_2'],how='inner')
third_merged

0       FAZIO FARMS OPERATING COMPANY, LLC
1                       CHARLIE SUNDERLAND
2                        MICHAEL RUDEBUSCH
3                             LODAHL FARMS
4                   DUNSON HARVESTING, INC
                       ...                
2715                     JAMES L SCHNELLER
2716                       STEPHEN G MYERS
2717                            ANDY POVEY
2718                SILVER CREEK SEED, LLC
2719                        HALABURA FARMS
Name: name_clean, Length: 2720, dtype: object

6                                ANNABELLA LAND & CATTLE
7                                    AUTUMN HILL ORCHARD
8                  CADDO CREEK RANCH, DBA PARADISE RANCH
12                           ROLLO FARM LABOR CONTRACTOR
14                                         SHARON MATHIS
                             ...                        
107                                         WALKER PLACE
108    COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...
110                                        JESUS LEDESMA
112          B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
113                                          DELIA ROJAS
Name: name_clean, Length: 114, dtype: object

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,Identifier
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,"Rafael Barajas, Sebring, Florida"


1 match


6                                ANNABELLA LAND & CATTLE
7                                    AUTUMN HILL ORCHARD
8                  CADDO CREEK RANCH, DBA PARADISE RANCH
12                           ROLLO FARM LABOR CONTRACTOR
14                                         SHARON MATHIS
                             ...                        
107                                         WALKER PLACE
108    COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...
110                                        JESUS LEDESMA
112          B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
113                                          DELIA ROJAS
Name: name_clean_2, Length: 114, dtype: object

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,name_clean_x,Name,"City, State",Violation,Duration,Start date,End date,Identifier,name_clean_y,name_clean_2
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,"Rafael Barajas, Sebring, Florida",RAFAEL BARAJAS,RAFAEL BARAJAS
1,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,SLASH EV RANCH,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,1 year,11/15/2014,11/14/2015,"Slash E.V. Ranch LLP, Rifle, CO",SLASH E.V. RANCH LLP,SLASH EV RANCH


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [464]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
pattern = r'(.+?)\s*\((COMPANY|INDIVIDUAL)\).*(AND)\s(.+?)\(INDIVIDUAL\)'


## your code here to apply it to the pos_example
match_pos = re.search(pattern, pos_example)
result_pos = [match_pos.group(1).strip(), match_pos.group(4).strip()] if match_pos else []
result_pos
## your code here to apply it to the negative example
match_neg = re.search(pattern, neg_example)
result_neg = [match_pos.group(1).strip(), match_neg.group(4).strip()] if match_neg else []
result_neg


['COUNTY FAIR FARM', 'ANDREW WILLIAMSON']

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here



   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here